In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelBinarizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

In [34]:
X = pd.read_csv('AwA2-features\Animals_with_Attributes2\Features\ResNet101\AwA2-features.txt', header=None, sep=" ", dtype = np.float64).values
Y = pd.read_csv('AwA2-features\Animals_with_Attributes2\Features\ResNet101\AwA2-labels.txt', header=None).values

lb = LabelBinarizer()
Y_bin = lb.fit_transform(Y)

df = pd.DataFrame(columns=['method','accuracy','precision', 'recall', 'f1_score'])

In [35]:
print("the format of X :" + str(X[0]))
print("the format of y: " + str(Y[0]))
print("the number of samples = " + str(X.shape[0]))
print("the number of samples X ?= Y: " + str(X.shape[0] == Y.shape[0]))
print("the featurn dimension of X = " + str(X.shape[1]))
print("the number of class: "  + str(Y[:,0].max() - Y[:,0].min() + 1))
print("the minimum class label:" + str(Y[:,0].min()))

the format of X :[0.12702841 3.2361083  0.93514752 ... 0.67223716 0.03226495 0.4076162 ]
the format of y: [1]
the number of samples = 37322
the number of samples X ?= Y: True
the featurn dimension of X = 2048
the number of class: 50
the minimum class label:1


In [36]:
#split the data to the 60% training set and 40% testing set, use array not dataframe
#X_train, X_test, Y_train, Y_test, Y_train_bin, Y_test_bin = train_test_split(X[:2000, :], Y[:2000], Y_bin[:2000, :], test_size=0.4, random_state=66)
X_train, X_test, Y_train, Y_test, Y_train_bin, Y_test_bin = train_test_split(X, Y, Y_bin, test_size=0.4, random_state=66)

print("X_train:" + str(X_train.shape))
print("Y_train:" + str(Y_train.shape))
print("Y_train_bin:" + str(Y_train_bin.shape))

X_train:(22393, 2048)
Y_train:(22393, 1)
Y_train_bin:(22393, 50)


In [37]:
# linear regression
from sklearn.linear_model import LinearRegression

#创建一个管道（Pipeline）实例
LinearRegression_pipeline = make_pipeline(StandardScaler(), LinearRegression())
LinearRegression_pipeline.fit(X_train, Y_train_bin)

joblib.dump(LinearRegression, 'LinearRegression.pkl')

predict_res = np.argmax(LinearRegression_pipeline.predict(X_test), axis = 1) + 1
LinearRegression_accuracy = accuracy_score(Y_test, predict_res)
LinearRegression_precision = precision_score(Y_test, predict_res, average='macro') #'micro', 'weighted'
LinearRegression_recalll = recall_score(Y_test, predict_res, average='macro')
LinearRegression_f1score = f1_score(Y_test, predict_res, average='macro')

print("Linear Regression accuracy: " + str(LinearRegression_accuracy))
print("Linear Regression precision: " + str(LinearRegression_precision))
print("Linear Regression recall: " + str(LinearRegression_recalll))
print("Linear Regression f1_score: " + str(LinearRegression_f1score))

df = df.append(pd.DataFrame([['Linear Regression', LinearRegression_accuracy, LinearRegression_precision, LinearRegression_recalll, LinearRegression_f1score]], columns=df.columns))


Linear Regression accuracy: 0.9182798579945073
Linear Regression precision: 0.9069363861159588
Linear Regression recall: 0.8606600897290966
Linear Regression f1_score: 0.8728671854912539


In [38]:
# kfold and Logistic Regression
from sklearn.linear_model import LogisticRegression

#choose the best penalty
# create a Pipeline instance
#LogisticRegression_pipeline_l1_ovr = make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', multi_class='ovr', solver='lbfgs', max_iter=100))
LogisticRegression_pipeline_l2_ovr = make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', multi_class='ovr', solver='lbfgs', max_iter=1000))
#LogisticRegression_pipeline_l1_mul = make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', multi_class='multinomial', solver='lbfgs', max_iter=100))
LogisticRegression_pipeline_l2_mul = make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', multi_class='multinomial', solver='lbfgs', max_iter=1000))
#pipeline = make_pipeline(LogisticRegression(multi_class='ovr'))
#scores_l1_ovr = cross_val_score(LogisticRegression_pipeline_l1_ovr, X=X_train, y=Y_train.ravel(), cv=10)
scores_l2_ovr = cross_val_score(LogisticRegression_pipeline_l2_ovr, X=X_train, y=Y_train.ravel(), cv=5)
#scores_l1_mul = cross_val_score(LogisticRegression_pipeline_l1_mul, X=X_train, y=Y_train.ravel(), cv=10)
scores_l2_mul = cross_val_score(LogisticRegression_pipeline_l2_mul, X=X_train, y=Y_train.ravel(), cv=5)

#print('l1_ovr Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores_l1_ovr),np.std(scores_l1_ovr)))
print('l2_ovr Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores_l2_ovr),np.std(scores_l2_ovr)))
#print('l1 Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores_l1_mul),np.std(scores_l1_mul)))
print('l2_mul Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores_l2_mul),np.std(scores_l2_mul)))

d:\IDE\Anaconda\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\IDE\Anaconda\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

l2_ovr Cross Validation accuracy: 0.922 +/- 0.003
l2_mul Cross Validation accuracy: 0.925 +/- 0.002


In [31]:
#choose the best logistic regression don't run 
LogisticRegression_pipeline_l2_mul.fit(X_train, Y_train.ravel())

predict_res = LogisticRegression_pipeline_l2_mul.predict(X_test)
LogisticRegression_accuracy = accuracy_score(Y_test, predict_res)
LogisticRegression_precision = precision_score(Y_test, predict_res, average='macro') #'micro', 'weighted'
LogisticRegression_recalll = recall_score(Y_test, predict_res, average='macro')
LogisticRegression_f1score = f1_score(Y_test, predict_res, average='macro')

print("Linear Regression accuracy: " + str(LogisticRegression_accuracy))
print("Linear Regression precision: " + str(LogisticRegression_precision))
print("Linear Regression recall: " + str(LogisticRegression_recalll))
print("Linear Regression f1_score: " + str(LogisticRegression_f1score))

df = df.append(pd.DataFrame([['Linear Regression', LinearRegression_accuracy, LinearRegression_precision, LinearRegression_recalll, LinearRegression_f1score]], columns=df.columns))

NameError: name 'LinearDiscriminantAnalysis_pipeline' is not defined

In [39]:
#linear discriminant analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LinearDiscriminantAnalysis_pipeline = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis(solver='svd'))
LinearDiscriminantAnalysis_pipeline.fit(X_train, Y_train.ravel())

joblib.dump(LinearDiscriminantAnalysis_pipeline, 'LinearDiscriminantAnalysis.pkl')

predict_res = LinearDiscriminantAnalysis_pipeline.predict(X_test)
LinearDiscriminantAnalysis_accuracy = accuracy_score(Y_test, predict_res)
LinearDiscriminantAnalysis_precision = precision_score(Y_test, predict_res, average='macro') #'micro', 'weighted'
LinearDiscriminantAnalysis_recalll = recall_score(Y_test, predict_res, average='macro')
LinearDiscriminantAnalysis_f1score = f1_score(Y_test, predict_res, average='macro')

print("Linear Discriminant accuracy: " + str(LinearDiscriminantAnalysis_accuracy))
print("Linear Discriminant precision: " + str(LinearDiscriminantAnalysis_precision))
print("Linear Discriminant recall: " + str(LinearDiscriminantAnalysis_recalll))
print("Linear Discriminant f1_score: " + str(LinearDiscriminantAnalysis_f1score))

df = df.append(pd.DataFrame([['Linear Discriminant Analysis', LinearDiscriminantAnalysis_accuracy, LinearDiscriminantAnalysis_precision, LinearDiscriminantAnalysis_recalll, LinearDiscriminantAnalysis_f1score]], columns=df.columns))

Linear Discriminant accuracy: 0.9188157277781499
Linear Discriminant precision: 0.9013561344691954
Linear Discriminant recall: 0.8876857061359066
Linear Discriminant f1_score: 0.8920958319561848


In [40]:
# kfold and SVM
from sklearn.svm import LinearSVC

svm_pipeline_ovr = make_pipeline(StandardScaler(), LinearSVC(penalty='l2', multi_class='ovr', dual='auto', max_iter=1000))
svm_pipeline_cram = make_pipeline(StandardScaler(), LinearSVC(penalty='l2', multi_class='crammer_singer', dual='auto', max_iter=1000))

scores_l2_ovr = cross_val_score(svm_pipeline_ovr, X=X_train, y=Y_train.ravel(), cv=5)
scores_l2_mul = cross_val_score(svm_pipeline_cram, X=X_train, y=Y_train.ravel(), cv=5)

print('l2_ovr Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores_l2_ovr),np.std(scores_l2_ovr)))
print('l2_mul Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores_l2_mul),np.std(scores_l2_mul)))

d:\IDE\Anaconda\anaconda\envs\tensorflow\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\IDE\Anaconda\anaconda\envs\tensorflow\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# choose the best svm
LinearDiscriminantAnalysis_pipeline.fit(X_train, Y_train.ravel())

predict_res = LinearDiscriminantAnalysis_pipeline.predict(X_test)
LinearDiscriminantAnalysis_accuracy = accuracy_score(Y_test, predict_res)
LinearDiscriminantAnalysis_precision = precision_score(Y_test, predict_res, average='macro') #'micro', 'weighted'
LinearDiscriminantAnalysis_recalll = recall_score(Y_test, predict_res, average='macro')
LinearDiscriminantAnalysis_f1score = f1_score(Y_test, predict_res, average='macro')

print("Linear Regression accuracy: " + str(LinearDiscriminantAnalysis_accuracy))
print("Linear Regression precision: " + str(LinearDiscriminantAnalysis_precision))
print("Linear Regression recall: " + str(LinearDiscriminantAnalysis_recalll))
print("Linear Regression f1_score: " + str(LinearDiscriminantAnalysis_f1score))

df = df.append(pd.DataFrame([['Linear Regression', LinearRegression_accuracy, LinearRegression_precision, LinearRegression_recalll, LinearRegression_f1score]], columns=df.columns))

In [ ]:
print(df)
df.to_excel('res.xlsx',index=False)

              method  accuracy  precision    recall  f1_score
0  Linear Regression    0.9775   0.967475  0.967692  0.967229
